In [17]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from datetime import datetime
from sklearn.linear_model import SGDClassifier
import sklearn.cross_validation
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import xgboost
from vowpalwabbit.sklearn_vw import VWClassifier
import langdetect as ld
#from difflib import SequenceMatcher
from fuzzywuzzy import fuzz

In [65]:
SOURCE = "../data/train3_updated.csv"
df = pd.read_csv(SOURCE, sep=',')

In [52]:
df.head()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ,weight,IsClick
0,7,0.005422,0,1000.0,3,3,0,2,5,15,0.250000,5,22,93,0,0,1,1
1,1,0.012844,0,77170.0,3,3,0,2,5,15,0.250000,5,22,66,0,0,1,0
2,1,0.002102,0,1790.0,3,2,0,2,5,21,0.192308,3,10,39,84,0,1,0
3,7,0.013889,0,3000.0,3,2,0,2,5,21,0.192308,3,10,48,84,0,1,1
4,1,0.033885,1,5470.0,3,4,0,1,1,7,0.125000,3,10,65,59,15,1,1


In [53]:
df.shape

(5150998, 18)

In [66]:
median_Price = df.Price.median()
df.Price.fillna(median_Price, inplace=True)

In [55]:
df.describe()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ,weight,IsClick
count,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06
mean,3.594031e+00,1.125062e-02,3.144672e-01,1.842276e+04,2.693287e+00,1.702706e+00,3.985022e-01,1.863855e+00,2.537840e+00,1.437796e+01,1.966040e-01,2.867462e+00,1.459326e+01,5.101086e+01,2.292443e+01,2.613030e+00,3.321266e+01,1.966040e-01
std,2.972405e+00,1.593798e-02,4.643034e-01,9.228073e+04,5.958805e-01,1.303716e+00,5.079587e-01,3.429443e-01,3.858880e+00,2.413330e+01,1.980426e-01,1.977061e+00,5.775873e+00,1.994528e+01,2.966364e+01,7.221760e+00,2.325436e+01,3.974304e-01
min,1.000000e+00,1.000000e-05,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,1.000000e+00,2.715000e-03,0.000000e+00,1.300000e+03,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,1.100000e+01,3.600000e+01,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,1.000000e+00,6.100000e-03,0.000000e+00,4.290000e+03,3.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,6.000000e+00,1.578947e-01,3.000000e+00,1.500000e+01,5.200000e+01,0.000000e+00,0.000000e+00,5.000000e+01,0.000000e+00
75%,7.000000e+00,1.299900e-02,1.000000e+00,1.330000e+04,3.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,1.600000e+01,3.333333e-01,5.000000e+00,1.900000e+01,6.600000e+01,4.200000e+01,0.000000e+00,5.000000e+01,0.000000e+00
max,7.000000e+00,1.000000e+00,1.000000e+00,1.040000e+07,3.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,1.070000e+02,5.960000e+02,1.000000e+00,6.000000e+00,2.300000e+01,9.700000e+01,1.980000e+02,1.620000e+02,5.000000e+01,1.000000e+00


In [56]:
df.dtypes

Position                 int64
HistCTR                float64
IsUserLoggedOn           int64
Price                  float64
SearchCategoryLevel      int64
reg_ads                  int64
highlighted_ads          int64
context_ads              int64
user_clicks_no           int64
user_non_clicks_no       int64
user_click_prob        float64
day_of_week              int64
hour                     int64
AdTitleSZ                int64
SearchParamsSZ           int64
SearchQuerySZ            int64
weight                   int64
IsClick                  int64
dtype: object

In [67]:
labels = df["IsClick"]
labels.head()

0    1
1    0
2    0
3    1
4    1
Name: IsClick, dtype: int64

In [70]:
# scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
# df.weight = scaler.fit_transform(df.weight.reshape(-1, 1))

df["weight"] = df["weight"].replace(to_replace=50, value=1.5)

In [71]:
weights = df["weight"]
weights.head()
weights.value_counts()

1.5    3386272
1.0    1764726
Name: weight, dtype: int64

In [72]:
df = df.drop(["IsClick"], axis=1)
df = df.drop(["weight"], axis=1)
df.head()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ
0,7,0.005422,0,1000.0,3,3,0,2,5,15,0.250000,5,22,93,0,0
1,1,0.012844,0,77170.0,3,3,0,2,5,15,0.250000,5,22,66,0,0
2,1,0.002102,0,1790.0,3,2,0,2,5,21,0.192308,3,10,39,84,0
3,7,0.013889,0,3000.0,3,2,0,2,5,21,0.192308,3,10,48,84,0
4,1,0.033885,1,5470.0,3,4,0,1,1,7,0.125000,3,10,65,59,15


In [73]:
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)

df.Price = scaler.fit_transform(df.Price.reshape(-1, 1))

# df.Position = scaler.fit_transform(inputs.Position.reshape(-1, 1))
# df.day_of_week = scaler.fit_transform(inputs.day_of_week.reshape(-1, 1))
# df.hour = scaler.fit_transform(inputs.hour.reshape(-1, 1))

df.AdTitleSZ = scaler.fit_transform(df.AdTitleSZ.reshape(-1, 1))
df.SearchParamsSZ = scaler.fit_transform(df.SearchParamsSZ.reshape(-1, 1))
df.SearchQuerySZ = scaler.fit_transform(df.SearchQuerySZ.reshape(-1, 1))

/Users/arzam/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()
/Users/arzam/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:9: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/Users/arzam/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/Users/arzam/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [46]:
# weights = scaler.fit_transform(weights.reshape(-1, 1))

/Users/arzam/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':


In [74]:
weights.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: weight, dtype: float64

In [76]:
df.describe()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ
count,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06
mean,3.594031e+00,1.125062e-02,3.144672e-01,1.771419e-03,2.693287e+00,1.702706e+00,3.985022e-01,1.863855e+00,2.537840e+00,1.437796e+01,1.966040e-01,2.867462e+00,1.459326e+01,5.258851e-01,1.157799e-01,1.612981e-02
std,2.972405e+00,1.593798e-02,4.643034e-01,8.873147e-03,5.958805e-01,1.303716e+00,5.079587e-01,3.429443e-01,3.858880e+00,2.413330e+01,1.980426e-01,1.977061e+00,5.775873e+00,2.056215e-01,1.498164e-01,4.457877e-02
min,1.000000e+00,1.000000e-05,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,2.715000e-03,0.000000e+00,1.250000e-04,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,1.100000e+01,3.711340e-01,0.000000e+00,0.000000e+00
50%,1.000000e+00,6.100000e-03,0.000000e+00,4.125000e-04,3.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,6.000000e+00,1.578947e-01,3.000000e+00,1.500000e+01,5.360825e-01,0.000000e+00,0.000000e+00
75%,7.000000e+00,1.299900e-02,1.000000e+00,1.278846e-03,3.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,1.600000e+01,3.333333e-01,5.000000e+00,1.900000e+01,6.804124e-01,2.121212e-01,0.000000e+00
max,7.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,1.070000e+02,5.960000e+02,1.000000e+00,6.000000e+00,2.300000e+01,1.000000e+00,1.000000e+00,1.000000e+00


## Logistic Regression with Stochastic Gradient Descent

In [ ]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

In [78]:
CLF_LogReg_SGF = SGDClassifier(loss="log")
# CLF_LogReg_SGF.fit(df, labels, sample_weight=weights)
CLF_LogReg_SGF.fit(df, labels)
print(CLF_LogReg_SGF)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)


In [34]:
# CLF_LogReg_SGF.predict_proba(df)
# CLF_LogReg_SGF.score(df, labels)

0.87293433233715101

In [79]:
sklearn.metrics.log_loss(labels, CLF_LogReg_SGF.predict_proba(df))

0.35866668209302599

## XGboost